# Overview

This notebook uses the latest version, 0.1.3 of the github repo, and thus allowing poetry to install perch-hoplite==1.0.0.dev. Whereas repo 0.1.1 has poetry install perch-hoplite==0.1.1

this notebook uses my setup, which uses poetry. It combines both the embeddings and the agile modelling notebooks I pull from the github repo

In [ ]:
!git clone https://github.com/MasBioCoding/perch-hoplite-quantiles.git